# Full Adder BDD Generator

This script generates a Binary Decision Diagram (BDD) for a formal model of an approximate full adder with variable bit width.


10.2021, Fabian Garber, Simon Howind, Kagan Özten, Martin Resetarits, Peter Traunmüller 

Contact: e1326142@student.tuwien.ac.at

In [2]:
#Import necessary libraries
from pyeda.inter import * # <--this is from the official wiki, don't judge

#GVmagic is a package to use the external GraphViz library inside python
%load_ext gvmagic

#to easily get some multidimensional arrays
import numpy

#from pprint import pprint

#Libraries needed for the image creation
import pydot
from PIL import Image
from numpy import *

## VHDL from Kagan

 -- Full adder (exact)
 
S <= A XOR B XOR Cin ;

Cout <= (A AND B) OR (Cin AND A) OR (Cin AND B) ;

-- Full adder INXA1 (Approx.)

s_inv_1 <= A XOR B XOR Cin ; -- Signal for binding S_a and Cout_a

S_a <= s_inv_1;

Cout_a <= NOT(s_inv_1) ;

## Approx. Full Adder

In [3]:
#Approximate full adder type INXA1:
def approx_full_adder_bit(a_in, b_in, carry_in):
    #cast to bool in case we've thrown in ints
    a_in = bool(int(a_in))
    b_in = bool(int(b_in))
    carry_in = bool(int(carry_in))
    
    s_out = (a_in ^ b_in ) ^ carry_in
    c_out = not s_out
    
    return s_out,c_out

#n-bit approx. full adder
def approx_full_adder(a_in, b_in, input_width):
    bin_format = "{0:0"+str(input_width)+"b}" #{0:08b}
    
    #format the input as string so we can pick single values
    a_in_bin_str = str(bin_format.format(a_in)) #"01101001"
    b_in_bin_str = str(bin_format.format(b_in)) #"01110101"
    
    #initialize vars for the loop
    carry_out = 0
    out = [0] * (input_width+1)
    
    #loop through each digit of the bin and use our approx adder
    for i in range(input_width, 0, -1):
        out[i],carry_out = approx_full_adder_bit(a_in_bin_str[i-1], b_in_bin_str[i-1], carry_out)
    
    #add the carry as leading digit
    out[0]=carry_out  

    return out
    
#run a quick test
#approx_full_adder(0b1111, 0b0000, 4)
print("approx:"+str(list(map(int, approx_full_adder(0b0011, 0b0000, 4)))))


approx:[0, 1, 0, 1, 1]


## Exact Full Adder

In [4]:
#exact full adder:
def exact_full_adder_bit(a_in, b_in, carry_in):
    #cast to bool in case we've thrown in ints
    a_in = bool(int(a_in))
    b_in = bool(int(b_in))
    carry_in = bool(int(carry_in))
    
    s_out = (a_in ^ b_in ) ^ carry_in
    c_out = (a_in & b_in ) | (a_in & carry_in ) | (carry_in & b_in )
    
    return s_out,c_out

#n-bit exact full adder
def exact_full_adder(a_in, b_in, input_width):
    bin_format = "{0:0"+str(input_width)+"b}" #{0:08b}
    
    #format the input as string so we can pick single values
    a_in_bin_str = str(bin_format.format(a_in)) #"01101001"
    b_in_bin_str = str(bin_format.format(b_in)) #"01110101"
        
    #initialize vars for the loop
    carry_out = 0
    out = [0] * (input_width+1)
    
    #loop through each digit of the bin and use our exact adder
    for i in range(input_width, 0, -1):
        out[i],carry_out = exact_full_adder_bit(a_in_bin_str[i-1], b_in_bin_str[i-1], carry_out)
    
    #add the carry as leading digit
    out[0]=carry_out  
    return out
    
#run a quick test
#exact_full_adder(0b1111, 0b0000, 4)
print("exact:"+str(list(map(int, exact_full_adder(0b0111, 0b0001, 4)))))

exact:[0, 1, 0, 0, 0]


## Running both adders

In [41]:
#test the adders with numbers

def run_adders(input_width, input_combinations, VERBOSE_output):


    #count the combinations 
    input_combination_nr = 0

    #string to hold a binary information if the approx adder was wrong -> "00111001010"; to be used with BDD later
    #1 == ERROR, 0==Correct
    bin_error = ""
    #2d error array to where the XY axis are inputs A and B and the values are the absolute error; to be used for image later
    abs_error_array = numpy.zeros(shape=(input_combinations, input_combinations, 3), dtype=numpy.uint8)
    #abs_error_array = numpy.zeros(shape=(input_combinations, input_combinations))

    #Loop through all combinations for input A
    for i in range(input_combinations):
        #Loop through all combinations for input B
        for j in range(input_combinations):
            #display the input combination as "A + B =" 
            bin_format = "{0:0"+str(input_width)+"b}" #{0:08b}
            if(VERBOSE_output):
                print(""+str(bin_format.format(i)), end =" + ")
                print(""+str(bin_format.format(j)), end =" = ")

            #Throw the numbers into the approximate adder
            approx_result = approx_full_adder(i, j, input_width)
            approx_result_int = 0
            #Format the output of the approximate adder and add it to the output thruth table
            for k in range(input_width+1):
                output_truth_table_approx[k][input_combination_nr] = approx_result[k]
                approx_result_int =  approx_result_int + int(approx_result[k]) * 2**(input_width-k)
            bin_format_plus1 = "{0:0"+str(input_width+1)+"b}" #{0:08b} 
            #Display the output of the approximate adder
            if(VERBOSE_output):
                print("approx:" + bin_format_plus1.format(approx_result_int), end =", ")

            #Throw the numbers into the exact adder
            exact_result = exact_full_adder(i, j, input_width)
            exact_result_int = 0
            #Format the output of the exact adder 
            for k in range(input_width+1):
                output_truth_table_exact[k][input_combination_nr] = exact_result[k]
                exact_result_int =  exact_result_int + int(exact_result[k]) * 2**(input_width-k)
            #Display the output of the exact adder
            if(VERBOSE_output):
                print("exact:" + bin_format_plus1.format(exact_result_int), end =", ")

            #Calculate the absolute error distance 
            #From Fig. 5a Formal_Methods_for_Exact_Analysis_of_Approximate_Circuits.pdf
            abs_error = abs(approx_result_int - exact_result_int)
            #abs_error_array[i][j][0] = abs_error #0=RED, 1=GREEN, 2=BLUE
            abs_error_array[i][j] = [255, 255-abs_error ,255-abs_error ] #0=RED, 1=GREEN, 2=BLUE
            if(abs_error > 0):
                bin_error = bin_error + "1"
            else:
                bin_error = bin_error + "0"
            if(VERBOSE_output):
                print("error: " + str(abs_error), end = " ")

            input_combination_nr = input_combination_nr + 1
            #print an newline 
            if(VERBOSE_output):
                print("Combination: "+ str(input_combination_nr) +"/" + str(input_combinations**2), end ="")
                print()
    return output_truth_table_approx, output_truth_table_exact, bin_error, abs_error_array
        

## Truth table from adder outputs

In [6]:
#takes the 2D array from the adder function and transforms it into strings that can be fed into the BDD

def generate_truth_table(output_truth_table_approx, output_truth_table_exact, bin_error, input_width, input_combinations, VERBOSE_output):
    #generate 1D string arrays
    output_truth_table_approx_str = [""] * (input_width+1)
    output_truth_table_exact_str = [""] * (input_width+1)


    #fill the 1D arrays with the row elements of the 2D arrays -> [0,1,0,1,1] shoud result in ["01011"]
    for h in range(input_width+1):
        for i in range(input_combinations**2):
            output_truth_table_approx_str[h] = output_truth_table_approx_str[h] + str(int(output_truth_table_approx[h][i]))
            output_truth_table_exact_str[h] = output_truth_table_exact_str[h] + str(int(output_truth_table_exact[h][i]))
    if(VERBOSE_output):
        print("Approx. output truth tables: " + str(output_truth_table_approx_str))
        print("Exact output truth tables: " + str(output_truth_table_exact_str))
        print("Error truth table (0=correct): " + bin_error)
    return output_truth_table_approx_str, output_truth_table_exact_str


## BDD table creation

In [7]:
#Create BDDs out of the ERROR truth table

def generate_BDD(input_truthtable, input_width, filename, VERBOSE_output):
    #create the inputs with their correct width
    A_in = exprvars('A', input_width)
    B_in = exprvars('B', input_width)

    #join both inputs so we again have something like A+B=Y
    X = A_in + B_in
    EDA_truth_table = truthtable(X, input_truthtable)
    EDA_expression = truthtable2expr(EDA_truth_table)
    EDA_bdd = expr2bdd(EDA_expression)
   
    #convert the BDD to a dot-thingie and then hacksy-macksy create a png
    EDA_error_bdd_dots = EDA_bdd.to_dot()
    (graph, ) = pydot.graph_from_dot_data(EDA_error_bdd_dots)
    graph.write_png(str(filename) + '.png')
    
        
    #if(VERBOSE_output):
    #    print("Binary Error BDD:")
    #    print("1==ERROR, 0==CORRECT")
    #    %dotobj EDA_bdd
    
    
    return EDA_bdd
    

## Visualization

In [64]:
def generate_image(abs_error_array, input_combinations, filename,  VERBOSE_output):
    
    #find maximum error:
    max_error = 0
    for f in range(input_combinations):
        for g in range(input_combinations):
            if(255-abs_error_array[f][g][2] > max_error):
                max_error = 255-abs_error_array[f][g][2]
                
    print("max error: "+ str(max_error))
    #scale colors to max error
    for f in range(input_combinations):
        for g in range(input_combinations):
            abs_error_array[f][g][1] = 255 - ((255-abs_error_array[f][g][1])/max_error) * 255 #scale green
            abs_error_array[f][g][2] = 255 - ((255-abs_error_array[f][g][2])/max_error) * 255 #scale blue
            
    if(VERBOSE_output):
        print("Error array scaled:" + str(abs_error_array))
        
    row_im = Image.fromarray(abs_error_array)
    row_im.save(str(filename) + ".png")

340282366920938463463374607431768211456

## RUN everything

In [66]:
#run all the functions after each other

#define the inputs
input_width = 8 #input width for the adders in bit
input_combinations = 2**input_width
VERBOSE_output = 0
    

#define some variables:

#Initialize an array for each output truth table
#each row is a complete truth table for a single output bit
output_truth_table_approx = numpy.zeros(shape=(input_width+1, input_combinations**2), dtype=numpy.uint8)
output_truth_table_exact = numpy.zeros(shape=(input_width+1, input_combinations**2), dtype=numpy.uint8)
#example with 2 input bits to visualize:
#[[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,]
#[1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,]
#[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,]]
#2 input bits result in 3 output bits and 16 input combinations. from 00+00=000 to 11+11=110

#run the adders with the defined bitwidth
output_truth_table_approx, output_truth_table_exact, bin_error, abs_error_array =\
run_adders(input_width, input_combinations, VERBOSE_output)

#create truth table string arrays
output_truth_table_approx_str, output_truth_table_exact_str =\
generate_truth_table(output_truth_table_approx, output_truth_table_exact, bin_error, input_width, input_combinations, VERBOSE_output)

#generate an error BDD and plot it
EDA_error_bdd =\
generate_BDD(bin_error, input_width, "images/error_bdd" , VERBOSE_output)



#generate approx output BDDs and plot them 
for h in range(input_width+1):
    EDA_approx_bdd =\
    generate_BDD(output_truth_table_approx_str[h], input_width, "images/BDD_approx_bit"+str(h) , VERBOSE_output)
#    %dotobj EDA_approx_bdd

#generate exact output BDDs and plot them 
for z in range(input_width+1):
    EDA_exact_bdd =\
    generate_BDD(output_truth_table_exact_str[z], input_width, "images/BDD_exact_bit"+str(z) , VERBOSE_output)
#    %dotobj EDA_exact_bdd

#generate an imgage with the absolute error 
#print(abs_error_array)
generate_image(abs_error_array.astype(numpy.uint8), input_combinations, "images/abs_error_visual" , VERBOSE_output)



max error: 254


## Stuff to look back at later

In [ ]:
#Generate BDD variables with the rigth width
input_A = bddvars('input_a', input_width)
input_B = bddvars('input_b', input_width)
carry_C = bddvars('carry_c', input_width)


for i in range(input_width):
    if i == 0: #catch the case for the "zeroth" bit
        approx_add_func_S= (input_A[i] ^ input_B[i]) ^ False
        approx_add_func_C = ~((input_A[i] ^ input_B[i]) ^ False)
        exact_add_func_S = (input_A[i] ^ input_B[i]) ^ False
        exact_add_func_C = (input_A[i] & input_B[i]) | (input_A[i] & False) | (False & input_B[i])
        
    else:
        approx_add_func_S= (input_A[i] ^ input_B[i]) ^ carry_C[i]
        approx_add_func_C = ~((input_A[i] ^ input_B[i]) ^ carry_C[i])
        exact_add_func_S = (input_A[i] ^ input_B[i]) ^ carry_C[i]
        exact_add_func_C = (input_A[i] & input_B[i]) | (input_A[i] & carry_C[i]) | (carry_C[i] & input_B[i])
        

approx_add_bdd_S = expr2bdd(approx_add_func_S)
approx_add_bdd_C = expr2bdd(approx_add_func_C)
%dotobj approx_add_bdd_S
%dotobj approx_add_bdd_C
